In [54]:
%pip install aif360[AdversarialDebiasing]
%pip install aif360[Reductions]

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Fairness Analysis with AIF360

In [55]:
import pandas as pd
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import ClassificationMetric
from aif360.algorithms.preprocessing import Reweighing
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score

In [56]:
df = pd.read_csv("compas-scores-two-years-preprocessed.csv")
df.head()

,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,days_in_jail,age_cat,score_text,decile_score,sex_Male,race_African-American,race_Asian,race_Caucasian,race_Hispanic,race_Native American,race_Other,c_charge_degree_M,type_of_assessment_Risk of Recidivism,two_year_recid
0,-0.936096,-0.148376,-0.183093,1.930558,0.071314,-0.064439,0.0,2.0,1.0,3.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.5
1,0.677988,-0.148376,-0.183093,-0.222566,-0.325854,-0.120018,0.0,0.0,1.0,0.0,0.5,0.0,0.0,1.0,0.0,0.0,0.0,0.5,1.0,0.0
2,0.762940,-0.148376,-0.183093,-0.222566,-0.723023,-0.052088,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.5,0.0,0.0,0.5,0.5,1.0,0.5
3,0.677988,-0.148376,-0.183093,-0.222566,-0.127270,-0.064439,0.0,0.0,1.0,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.5,1.0,1.0
4,0.338181,-0.148376,-0.183093,-0.222566,-0.723023,-0.064439,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [57]:

# Split the data into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [58]:
# Separate features and labels
X_train = train_df.drop(columns=['two_year_recid'])
y_train = train_df['two_year_recid'].astype('int')
X_test = test_df.drop(columns=['two_year_recid'])
y_test = test_df['two_year_recid'].astype('int')

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [59]:
# Train a logistic regression model
model = LogisticRegression()
model.fit(X_train_scaled, y_train)

LogisticRegression()

In [60]:
# Make predictions
y_pred = model.predict(X_test_scaled)
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision_score(y_test, y_pred):.2f}")

Accuracy: 0.68
Precision: 0.61


In [61]:

# Map the labels in the dataset to 1 (favorable) and 0 (unfavorable)
train_df['two_year_recid'] = train_df['two_year_recid'].apply(lambda x: 1 if x == 1.0 else 0)
test_df['two_year_recid'] = test_df['two_year_recid'].apply(lambda x: 1 if x == 1.0 else 0)

In [62]:
# Convert the dataset to a BinaryLabelDataset
def convert_to_binary_label_dataset(df, label_name, protected_attribute_names):
    dataset = BinaryLabelDataset(
        df=df,
        label_names=[label_name],
        protected_attribute_names=protected_attribute_names,
        favorable_label=1,
        unfavorable_label=0
    )
    return dataset

In [63]:
# Convert the training and testing data to BinaryLabelDataset
protected_attribute_names = ['race_African-American','race_Asian','race_Caucasian', 'race_Hispanic', 'race_Native American', 'race_Other' ]
train_dataset = convert_to_binary_label_dataset(train_df, 'two_year_recid', protected_attribute_names)
test_dataset = convert_to_binary_label_dataset(test_df, 'two_year_recid', protected_attribute_names)
# Define unprivileged and privileged groups
unprivileged_groups = [{'race_African-American': 1.0}]
privileged_groups = [{'race_Caucasian': 1.0}]

In [64]:
# Apply reweighing
reweighing = Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
reweighing.fit(train_dataset)
train_dataset_transformed = reweighing.transform(train_dataset)

In [65]:
# Train a logistic regression model on the reweighted dataset
model_reweighted = LogisticRegression()
model_reweighted.fit(
    train_dataset_transformed.features,
    train_dataset_transformed.labels.ravel(),
    sample_weight=train_dataset_transformed.instance_weights
)

LogisticRegression()

In [66]:
# Make predictions on the test set
y_pred_reweighted = model_reweighted.predict(test_dataset.features)
# Evaluate the model
accuracy_reweighted = accuracy_score(test_dataset.labels, y_pred_reweighted)
print(f"Accuracy after reweighing: {accuracy_reweighted:.2f}")
print(f"Precision after reweighing: {precision_score(test_dataset.labels, y_pred_reweighted):.2f}")


Accuracy after reweighing: 0.65
Precision after reweighing: 0.40


Accuracy : Acceptable, but could be improved

Precision : Many false positives – potential concern

In [67]:
from aif360.datasets import BinaryLabelDataset

pred_dataset = test_dataset.copy()
pred_dataset.labels = y_pred_reweighted.reshape(-1, 1)

metric = ClassificationMetric(test_dataset, pred_dataset,
                              unprivileged_groups=unprivileged_groups,
                              privileged_groups=privileged_groups)
metric = ClassificationMetric(test_dataset, test_dataset, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
print(f"Statistical Parity Difference: {metric.statistical_parity_difference():.2f}")
print(f"Disparate Impact: {metric.disparate_impact():.2f}")

Statistical Parity Difference: 0.12
Disparate Impact: 1.37


Statistical Parity Diff	: Mild disparity in favor of privileged group

Disparate Impact : Favorable outcomes skewed toward unprivileged – possibly overcorrected

### Post-processing: Reject Option Classification (ROC)

In [ ]:
#technique to improve fairness
from aif360.algorithms.postprocessing import RejectOptionClassification
from aif360.datasets import BinaryLabelDataset

# Convert predictions to BinaryLabelDataset
test_pred = test_dataset.copy()
test_pred.labels = y_pred_reweighted

# Apply ROC
roc = RejectOptionClassification(
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
    low_class_thresh=0.01,
    high_class_thresh=0.99,
    num_class_thresh=100,
    num_ROC_margin=50,
    metric_name="Statistical parity difference",
    metric_ub=0.05,  # Upper bound for SPD
    metric_lb=-0.05  # Lower bound for SPD
)

roc = roc.fit(test_dataset, test_pred)
test_pred_roc = roc.predict(test_pred)

# Evaluate fairness
metric_roc = ClassificationMetric(test_dataset, test_pred_roc,
                                   unprivileged_groups=unprivileged_groups,
                                   privileged_groups=privileged_groups)
print(f"SPD after ROC: {metric_roc.statistical_parity_difference():.3f}")
print(f"DI after ROC: {metric_roc.disparate_impact():.3f}")


SPD after ROC: 0.120
DI after ROC: 1.367


/home/estellelpb/mosig/Fairness_COMPAS/.venv/lib/python3.12/site-packages/aif360/algorithms/postprocessing/reject_option_classification.py:160: UserWarning: Unable to satisy fairness constraints
  warn("Unable to satisy fairness constraints")
